In [1]:
# İmports
import requests
from bs4 import BeautifulSoup
from IPython.display import display, Markdown
import ollama


In [10]:
MODEL = "llama3.2"


In [2]:
# class Website:
#     url: str
#     title: str
#     text: str

#     def __init__(self, url):
#         self.url = url
#         response = requests.get(url)
#         soup=BeautifulSoup(response.content, "html.parser")
#         self.title = soup.title.string if soup.title else "No Title Found"
#         for irrelevant in soup.body(['script', 'style', 'header', 'footer', 'nav', 'aside']):
#             irrelevant.decompose()
#         self.text = soup.body.get_text(separator="/n",strip=True)  # Limit to first 2000 characters    

In [3]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

class Website:
    def __init__(self, url):
        self.url = url
        self.response = requests.get(url)
        self.soup = BeautifulSoup(self.response.content, "html.parser")

        # Temel bilgiler
        self.title = self.soup.title.string if self.soup.title else "No Title Found"
        self.meta_description = self.soup.find("meta", attrs={"name": "description"})
        self.meta_description = self.meta_description["content"] if self.meta_description else "No Description"
        self.meta_keywords = self.soup.find("meta", attrs={"name": "keywords"})
        self.meta_keywords = self.meta_keywords["content"] if self.meta_keywords else "No Keywords"
        
        # H1-H6 başlıklar
        self.headings = {f"h{i}": [h.get_text(strip=True) for h in self.soup.find_all(f"h{i}")] for i in range(1,7)}
        
        # Body text (temizlenmiş)
        if self.soup.body:
            for irrelevant in self.soup.body(['script', 'style', 'header', 'footer', 'nav', 'aside']):
                irrelevant.decompose()
            self.text = self.soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        
        # Linkler
        self.links = []
        for a in self.soup.find_all('a', href=True):
            self.links.append(urljoin(self.url, a['href']))
        
        # Görseller ve alt metinleri
        self.images = []
        for img in self.soup.find_all('img'):
            src = urljoin(self.url, img.get('src', ''))
            alt = img.get('alt', '')
            self.images.append({'src': src, 'alt': alt})
        
        # Kelime sayısı
        self.word_count = len(self.text.split())

    def summary(self):
        return {
            "url": self.url,
            "title": self.title,
            "meta_description": self.meta_description,
            "meta_keywords": self.meta_keywords,
            "headings": self.headings,
            "word_count": self.word_count,
            "links_count": len(self.links),
            "images_count": len(self.images)
        }


In [17]:
deponext=Website("https://deponext.com.tr/")
deponext_summary = deponext.summary()
deponext_summary

{'url': 'https://deponext.com.tr/',
 'title': 'Anasayfa | Depo Next - Güvenli Depolama Çözümleri',
 'meta_description': 'Depo Next, eşyalarınızı güvenle ve kolaylıkla depolamanız için modern ve esnek depolama alanları sunar. İhtiyacınıza uygun, ekonomik ve güvenli depolama çözümleri ile size en iyi hizmeti vermeyi amaçlıyoruz.',
 'meta_keywords': 'Depo Next,güvenli depo,eşya depolama,kolay depo kiralama,esnek depolama alanları,depola,depo hizmetleri',
 'headings': {'h1': ['DepoNext'],
  'h2': ['Eşyalarınız İçin Güvenli ve Kolay Depolama Çözümü',
   'İLETİŞİM',
   'Merkezî Konumda, Kolay Erişim!',
   'Müşterilerimizin güvenini kazandık',
   'Müşteri Yorumları',
   'Sizler içinher şeyi düşündük.Sizrahat bir nefes alabilirsiniz.',
   'Depolarımızın Bulunduğu İstanbul İlçeleri',
   'Sıkça Sorulan Sorular',
   'DepoNext'],
  'h3': ['Beklentiler dinlenir,',
   'operasyon hakkında bilgilendirilir.',
   'İhtiyaca uygun,',
   'teklif verilir.',
   'Eşyalar taşınmaya ve,',
   'depolanmaya hazır.

In [5]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [6]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [7]:

# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [11]:
def summarize_website(url):
    website=Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    display(Markdown(response['message']['content']))

In [12]:
summarize_website("https://deponext.com.tr/")

# Anasayfa | Depo Next - Güvenli Depolama Çözümleri

**Kısa Özet**

DepoNext, Istanbul'da bulunan bir depolama şirketidir. Kendinize özel depolama alanlarını oluşturmak ve eşylerinizi güvenli ve kolayca depolamak isteyen personasalla hizmet vermektedir.

### Haberler ve Bilgiler

*   **Eşyalarınız İçin Güvenli ve Kolay Depolama Çözümü**: Kendinize özel depolama alanınızı oluşturun ve Eşyelerinize güvenli bir şekilde depolama hizmeti alabilirsiniz.
*   **Depo Kiralama**: İhtiyaca uygun, teklif verilir. Depolanacak eşyaların veya ürünlerin durumu göz önünde bulundurularak teklif verilir.
*   **Eşyalarınız Size Özel, Oda Deponuzda Güvende**: Eşyaları size özel bir alanda yerleştirildi ve güvende.
*   **ADRES**: Gayrettepe, Vefabey Sokağı 25/1, 34349 Beşiktaş/İstanbul
*   **İLETİŞİM**: info@deponext.com, +90 537 323 43 43
*   **Harita'dan Gör**

### Merak Edilen Sorular

*   Eşyalarınız nasıl taşınıyor?
*   Depolama süresi ne kadar olabilir?
*   Depo alanları güvenli mi?
*   Depolama ücretleri nasıl belirleniyor?
*   Depolanmış eşyaları tekrar nasıl alabilirim?

### Hizmetler

*   Kök Depolama
*   Koli Depolama
*   Yürükli Koli Depolama
*   Taşıma
*   Paketleme
*   Konteyner Depolama

In [13]:
def display_summary(url):
    summary=summarize_website(url)
    display(Markdown(summary))

In [14]:
display_summary("https://deponext.com.tr/")

# DepoNext Anasayfa
## Güvenli ve Kolay Depolama Çözümü

DepoNext, Türkiye'de binlerce kişi ve işletmeye güvenli depolama ve taşıma hizmetleri sunuyor. 

### Başarılar
- 5.000+ active storage alanları
- 12.000+ aktif depo alanı (m²)
- %98+ müşteri memnuniyeti

### Hizmetler
- Kişiye özel depolama alanları
- Eşyalarınızı güvenli ve modern bir şekilde saklayın
- Erişim kolaylığı ve şehir merkezleri nearness

### Müşteri Yorumları
- "Eşyalarımız zamanında alındı ve profesyonelce depoya yerleştirildi. Her şey mükemmeldi!" - Ayşe Yılmaz (İstanbul)
- "Depolama süreci boyunca her aşamada bilgilendirildim. Çok güven verici bir hizmet." - Mehmet Kara (Ankara)
- "Taşınma stresini tamamen ortadan kaldırdılar. Ekibin ilgisi ve profesyonelliği harikaydı." - Kasım Demir (İzmir)

### Güvenli Lokasyon
- İstanbul'un kalbinde yer alan Gayrettepe Mahallesi, Vefabey Sokak, Say Apartmani B Blok No:25 İç Kapı No:1, Beşiktaş

### İletişim
- Adres: info@deponext.com, +90 537 323 43 43
- Harita'dan gör

<IPython.core.display.Markdown object>